In [1]:
import os
%pwd

'c:\\Users\\user\\Desktop\\poultry_diseases\\Chicken-Disease-Classifier\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\user\\Desktop\\poultry_diseases\\Chicken-Disease-Classifier'

In [3]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [5]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self)->EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config



In [6]:
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [7]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-01-30 14:20:49,473 INFO common yaml file: config\config.yaml loaded successfully]
[2024-01-30 14:20:49,487 INFO common yaml file: params.yaml loaded successfully]
[2024-01-30 14:20:49,493 INFO common created directory at: artifacts]
Found 116 images belonging to 2 classes.

[2024-01-30 14:20:53,391 WARNING module_wrapper From c:\Users\user\Desktop\poultry_diseases\Chicken-Disease-Classifier\dl\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
8/8 [==============================] - 62s 7s/step - loss: 0.7290 - accuracy: 0.5000
[2024-01-30 14:21:54,381 INFO common json file saved at: scores.json]
